In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import LSTM, Dense, SimpleRNN, Input, Activation, Dropout
from keras import backend as K
from tensorflow.keras.optimizers import Adam,SGD
import tensorflow as tf
from keras.models import Model
from keras.models import load_model
import time #to calculate the computation time

In [ ]:
# specifying constant parameters
#input varioable: Q, C_A0
#state variable: T, C_A

T_0 = 300
V = 1
k_0 = 8.46*(np.power(10,6))
C_p = 0.231
rho_L = 1000
F = 5
E = 5*(np.power(10,4))
delta_H = -1.15*(np.power(10,4))
R = 8.314

C_A0s = 4   # the steady state for input variable C_A0
Q_s = 0.0  # the steady state for input variable Q

C_As = 1.95 # the steady state for state variable C_A
T_s = 402  # the steady state for state variable T

t_final = 0.01  #the control period
t_step = 1e-4   # the step to use first-principle to calculate the state
P = np.array([[1060, 22], [22, 0.52]])

In [ ]:
# generating inputs and initial states for CSTR, all expressed in deviation form

# Test 1 full stability region
u1_list = np.linspace(-3.5, 3.5, 20, endpoint=True)
u2_list = np.linspace(-5e5, 5e5, 20, endpoint=True)
T_initial = np.linspace(300, 550, 20, endpoint=True) - T_s
CA_initial = np.linspace(1, 4., 20, endpoint=True) - C_As    # CA and T must be >0
#print(T_initial)
#control variable: C_A0, Q
#state variable: C_A, T

# sieve out initial states that lie outside of stability region

T_start = list()
CA_start = list()

for T in T_initial:   
    for CA in CA_initial:
        x = np.array([CA, T])
        if x @ P @ x < 368:   #calculate the stability region for state 
            CA_start.append(CA)
            T_start.append(T)
print("number of initial conditions: {}".format(len(CA_start)))

# convert to np.arrays
CA_start = np.array([CA_start])
T_start = np.array([T_start])
print(CA_start.shape)
x_deviation = np.concatenate((CA_start.T, T_start.T), axis=1)  # every row is a pair of initial states within stability region
print("shape of x_deviation is {}".format(x_deviation.shape))
print(x_deviation.shape)  # the initial state is in 

In [ ]:
def CSTR_simulation(F, V, C_A0, k_0, E, R, T_0, delta_H, rho_L, C_p, Q, t_final, t_step, C_A_initial, T_initial):
    """
        simulating CSTR using forward Euler method
    """
    
    C_A_list = list()  # evolution of CA over time
    T_list = list()  # evolution of T over time
    
    C_A = C_A_initial + C_As  # the real state.the derivation plus the steady state
    T = T_initial + T_s
    
    for i in range(int(t_final / t_step)):
        dCAdt = F / V * (C_A0 - C_A) - k_0 * np.exp(-E / (R * T)) * C_A**2
        dTdt = F / V * (T_0 - T) - delta_H / (rho_L * C_p) * k_0 * np.exp(-E / (R * T)) * C_A**2 + Q / (rho_L * C_p * V)
        
        C_A += dCAdt * t_step
        T += dTdt * t_step
        
        if i%5 ==0:
            C_A_list.append(C_A - C_As)  # in deviation form
            T_list.append(T - T_s)  # in deviation form 
    
    return C_A_list, T_list

In [ ]:
# get X and y data for training and testing

CA_output = list()
T_output = list()

CA_input = list()
T_input = list()
CA0_input = list()
Q_input = list()   #input variable for 

for u1 in u1_list:
    C_A0 = u1 + C_A0s
    
    for u2 in u2_list:
        Q = u2 + Q_s
        
        for C_A_initial, T_initial in x_deviation:
            CA0_input.append(u1)
            Q_input.append(u2)
            CA_input.append(C_A_initial)
            T_input.append(T_initial)
            
            C_A_list, T_list = CSTR_simulation(F, V, C_A0, k_0, E, R, T_0, delta_H, rho_L, C_p, Q, t_final, t_step, C_A_initial, T_initial)
            CA_output.append(C_A_list)
            T_output.append(T_list)

In [ ]:
# collate input for RNN

CA0_input = np.array(CA0_input)
CA0_input = CA0_input.reshape(-1,1,1)

Q_input = np.array(Q_input)
Q_input = Q_input.reshape(-1,1,1)

CA_input = np.array(CA_input)
CA_input = CA_input.reshape(-1,1,1)

T_input = np.array(T_input)
T_input = T_input.reshape(-1,1,1)

RNN_input = np.concatenate((T_input, CA_input, Q_input, CA0_input), axis=2)   #the value for input variable and the initial value for state variable 

"""
    the input to RNN is in the shape [number of samples x timestep x variables], and the input variables are same for every
    time step, not sure if my treatment here is correct
"""
print("RNN_input shape is {}".format(RNN_input.shape))
RNN_input = RNN_input.repeat(20, axis=1)  # to keep consensus with the shape for RNN_output, since the output variable is collected 100(0.01/1e-4) times for each RNN_input
print("RNN_input shape is {}".format(RNN_input.shape))

In [ ]:
# collate output for RNN

CA_output = np.array(CA_output)
CA_output = CA_output.reshape(-1, 20, 1)

T_output = np.array(T_output)
T_output = T_output.reshape(-1, 20, 1)

RNN_output = np.concatenate((T_output, CA_output), axis=2)
print("RNN_output shape is {}".format(RNN_output.shape))  # output shape: number of samples x timestep x variables

In [ ]:
# split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(RNN_input, RNN_output, test_size=0.3, random_state=123)

# define scalers for both X and y base on training data only
scaler_X = preprocessing.StandardScaler().fit(X_train.reshape(-1, 4))
scaler_y = preprocessing.StandardScaler().fit(y_train.reshape(-1, 2))


X_train = scaler_X.transform(X_train.reshape(-1, 4)).reshape(-1,20,4)
X_test = scaler_X.transform(X_test.reshape(-1, 4)).reshape(-1,20,4)
y_train = scaler_y.transform(y_train.reshape(-1,2)).reshape(-1,20,2)
y_test = scaler_y.transform(y_test.reshape(-1,2)).reshape(-1,20,2)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
model = Sequential()
model.add(SimpleRNN(32, activation='tanh', return_sequences=True))
model.add(SimpleRNN(32, activation='tanh', return_sequences=True))
model.add(Dense(2, activation='linear'))
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

t0 = time.time()
history = model.fit(X_train, y_train, epochs=300, batch_size=256, validation_split=0.25, verbose=2)
t1 = time.time()
print(t1-t0)

In [ ]:
#use the test data to evaluate the model
loss_and_metrics = model.evaluate(X_test, y_test, batch_size=256)
print(loss_and_metrics)

print(t1-t0)
model.save('model_n.h5')

In [ ]:
#load the basic model trained before, one hidden layer with 32 ne
basic = load_model('basic.h5')

In [ ]:
#use the test data to evaluate the model the generalization ability
loss_and_metrics = basic.evaluate(X_test, y_test, batch_size=256)
print(loss_and_metrics)


In [ ]:
#copy the basic model
basic_u1 = tf.keras.models.clone_model(basic)
basic_u1.set_weights(basic.get_weights())

inputs = tf.keras.Input(shape=(20,4))
basic_u1.layers[0].trainable = False
x = basic_u1.layers[0](inputs)
x1 = tf.keras.layers.SimpleRNN(32, activation='tanh', return_sequences=True)(x)
x2 = tf.keras.layers.Dense(2, activation='linear')(x1)

transf_model1 = tf.keras.Model(inputs=inputs,outputs=x2)

In [ ]:
print(transf_model1.summary())

In [ ]:
transf_model1.compile(optimizer='adam', loss='mse', metrics=['mse'])
t0 = time.time()
history1 = transf_model1.fit(X_train, y_train, epochs=150, batch_size=256, validation_split=0.25, verbose=2)
t1 = time.time()

print(t1-t0)

In [ ]:
#use the test data to evaluate the model
loss_and_metrics = transf_model1.evaluate(X_test, y_test, batch_size=256)
print(loss_and_metrics)

print(t1-t0)

In [ ]:
#unfreeze the basic model and do the fine-tuning for the whole Model
transf_model1.layers[1].trainable = True
print(transf_model1.summary())

#copy the basic model
transf_model1_F = tf.keras.models.clone_model(transf_model1)
transf_model1_F.set_weights(transf_model1.get_weights())

transf_model1_F.compile(optimizer='adam', loss='mse', metrics=['mse'])
print(transf_model1_F.summary())

In [ ]:
t0 = time.time()
history1 = transf_model1_F.fit(X_train, y_train, epochs=300, batch_size=256, validation_split=0.25, verbose=2)
t1 = time.time()

print(t1 - t0) #memory the computation time

loss_and_metrics = transf_model1_F.evaluate(X_test, y_test, batch_size=128)
print(loss_and_metrics)

In [ ]:
print(t1 - t0) #memory the computation time

loss_and_metrics = transf_model1.evaluate(X_test, y_test, batch_size=128)
print(loss_and_metrics)

In [ ]:
#calculate the input and the output

print(scaler_X.mean_)
print(scaler_X.scale_)

print(scaler_y.mean_)
print(scaler_y.scale_)

In [ ]:
transf_model1.save('transfer_model1.h5')

In [ ]:
loss_and_metrics = transf_model1.evaluate(X_test, y_test, batch_size=256)
print(loss_and_metrics)